# How to use the lookup file

This documentation runs through how to use the`lad_hmcts_region_lookup.csv` lookup to create shapefiles, geojson and topojson for use in mapping and other geospacial analysis

HMCTS regions are just groups of local authorities.  `lad_hmcts_region_lookup.csv` contains the correspondence between the two.

To create shapefiles of the regions, we just need to merge local authorities into wider regions.  To do this we will use the following workflow:

- Local authority shapefiles from ONS or Ordnance Survey -> PostGIS database table
- `lad_hmcts_region_lookup.csv` -> PostGIS database table
- Spatial SQL statement to merge local authorities into a new table containing hmcts regions
- Export from Postgres to HMCTS regions shapefiles
- Convert HMCTS regions to geojson and simplify using `ogr2ogr`
- Convert geojson to topojson using `topojson`

It is possible that the lookup may need to be updated in future.  It uses Local Authority District definitions from 2016.  If these are updated in future, the lookup may need to be updated.

Source data of Local Authority Disticts (LADs): Ordnance Survey Boundary Line https://www.ordnancesurvey.co.uk/opendatadownload/products.html#BDLINE
Alternative source data: http://geoportal.statistics.gov.uk/datasets/local-authority-districts-december-2016-full-clipped-boundaries-in-great-britain

Source data from HMCTS: https://jac.judiciary.gov.uk/sites/default/files/sync/application_selection/map-uk-courts-regions-hmcts-march2012.pdf  

In the following scripts we assume these are saved and unzipped to the working directory.

By overlaying the HMCTS pdf above on a map of the local authorities, we were able to produce the lookup. 

See the image [here](https://github.com/moj-analytical-services/lookup_hmcts_regions/blob/master/pics/map-uk-courts-regions-hmcts-march2012.png) for proof that the lookup is correct.

In [ ]:
import pandas as pd

In [ ]:
# Load lookup
lad_hmcts_region_lookup = pd.read_csv("lad_hmcts_region_lookup.csv", encoding="utf-8")

In [ ]:
Write shapefiles to postgres database

In [ ]:
%%bash 
shp2pgsql -I -s 27700 lad/Local_Authority_Districts_December_2016_Full_Clipped_Boundaries_in_Great_Britain.shp public.las_ons | psql -d postgres

In [ ]:
%%bash 
shp2pgsql -I -s 27700 bdline/Data/GB/district_borough_unitary_ward_region.shp public.las_os | psql -d postgres

In [ ]:
import psycopg2
con_string = "host='localhost' dbname='postgres' user='robinlinacre' password=''"
conn = psycopg2.connect(con_string)
cursor = conn.cursor()

from sqlalchemy import create_engine
engine = create_engine('postgresql://robinlinacre:@localhost:5432/postgres') 
lad_hmcts_region_lookup.to_sql("lad_hmcts_region_lookup", engine, index=False, if_exists="replace")

In [ ]:
# Spatial query that aggregates LAs to create regions
sql = """
create table hmcts_regions as
select st_union(ST_snaptogrid(geom,0.0001)), hmcts_region
from las_ons as shp
left join lad_hmcts_region_lookup as l
on shp.lad16cd = l.lad16cd_code
group by hmcts_region
"""
cursor.execute(sql)
conn.commit()

Write postgres data to shapefile on disk

In [ ]:
%%bash
pgsql2shp -f shapefiles/hmcts_regions -h localhost -u robinlinacre postgres "SELECT * from hmcts_regions"

Convert shapefiles into geojson and topojson

In [ ]:
%%bash
ogr2ogr -t_srs WGS84  -f GeoJSON   geojson/hmcts_regions.geojson   shapefiles/hmcts_regions.shp -simplify 100
topojson -o topojson/hmcts_regions.topojson geojson/hmcts_regions.geojson --simplify-proportion 0.04 -p